<a href="https://colab.research.google.com/github/yuracode/hsprogramming/blob/main/0.%E5%AE%9F%E8%A3%85%E3%83%87%E3%83%A2%E3%83%B3%E3%82%B9%E3%83%88%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision -qq

In [ ]:
import gradio as gr
from diffusers import DiffusionPipeline

# モデルの読み込み（初回は時間がかかります）
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base")
pipe = pipe.to("cuda")

def generate_image(prompt):
    image = pipe(prompt).images[0]
    return image

# Gradioインターフェースの作成
iface = gr.Interface(fn=generate_image,
                     inputs="text",
                     outputs="image",
                     title="AIイラストレーター",
                     description="好きな言葉（プロンプト）を入力して、画像を作ってみよう！")
iface.launch(share=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fe0d1742b18c821328.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 必要なライブラリをすべてインポート
import gradio as gr
from PIL import Image, ImageFilter, ImageDraw
import numpy as np
import torch
import torchvision.transforms.functional as F
from torchvision.models import resnet50
import requests

# --- AIモデルとラベルの準備 ---
# この準備は一度だけ行います
try:
    model = resnet50(pretrained=True)
    model.eval()

    response = requests.get("https://git.io/JJkYN")
    labels = response.text.split("\n")
    print("AIモデルの準備が完了しました。")
except Exception as e:
    print(f"AIモデルの準備中にエラーが発生しました: {e}")
    model = None
    labels = None

# --- 1. 画像加工のロジック（前回のコードと同じ） ---
def image_processing(img, filter_type, blur_level, watermark_text):
    if img is None:
        return None, "画像をアップロードしてください。"
    processed_img = img.copy()
    if filter_type == "白黒":
        processed_img = processed_img.convert('L')
    elif filter_type == "90度回転":
        processed_img = processed_img.rotate(90, expand=True)
    if blur_level > 0:
        processed_img = processed_img.filter(ImageFilter.GaussianBlur(radius=blur_level))
    if watermark_text:
        draw = ImageDraw.Draw(processed_img)
        draw.text((10, 10), watermark_text, fill="white", stroke_width=1, stroke_fill="black")
    return processed_img, f"処理完了：{filter_type}"

# --- 2. AIによる画像分析のロジック ---
# (教材のコードを関数として整理)
@torch.no_grad()
def predict(input_img):
    if model is None or labels is None or input_img is None:
        return "AIモデルが準備できていないか、画像がありません。"

    # PIL ImageをPyTorchが扱えるTensorに変換
    img = F.resize(input_img, (224, 224))
    img = F.to_tensor(img)
    if img.shape[0] == 1: # 白黒画像の場合
        img = img.repeat(3, 1, 1)
    img = F.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img = img.unsqueeze(0)

    # 推論の実行
    output = model(img).squeeze(0)
    probs = torch.nn.functional.softmax(output, dim=0).numpy()

    # 結果を{ラベル: 確率}の辞書形式で返す
    return {labels[i]: float(probs[i]) for i in range(1000)}

# --- GradioのUIを構築 ---
with gr.Blocks() as demo:
    gr.Markdown("# 総合画像処理アプリ")
    gr.Markdown("手動での画像加工と、AIによる画像分析ができます。")

    with gr.Tabs():
        # --- タブ1: 画像加工 ---
        with gr.TabItem("画像加工"):
            with gr.Row():
                with gr.Column():
                    proc_input = gr.Image(type="pil", label="加工したい画像をアップロード")
                    filter_radio = gr.Radio(["なし", "白黒", "90度回転"], label="フィルターを選択", value="なし")
                    blur_slider = gr.Slider(0, 10, value=0, label="ぼかしの強さ")
                    watermark_input = gr.Textbox(label="透かし文字")
                    proc_btn = gr.Button("画像処理を実行")
                with gr.Column():
                    proc_output = gr.Image(label="加工後の画像")
                    proc_status = gr.Textbox(label="処理ステータス")
            proc_btn.click(
                fn=image_processing,
                inputs=[proc_input, filter_radio, blur_slider, watermark_input],
                outputs=[proc_output, proc_status]
            )

        # --- タブ2: AI画像分析 ---
        with gr.TabItem("AI画像分析"):
            with gr.Row():
                with gr.Column():
                    ai_input = gr.Image(type="pil", label="分析したい画像をアップロード")
                    ai_btn = gr.Button("AI分析を実行")
                with gr.Column():
                    ai_output = gr.Label(num_top_classes=5, label="分析結果")
            ai_btn.click(
                fn=predict,
                inputs=ai_input,
                outputs=ai_output
            )

# 必要なライブラリのインストールを促すメッセージ
print("\nもし 'torch' や 'torchvision' がないというエラーが出たら、!pip install torch torchvision を実行してください。")
# アプリケーションの起動
demo.launch()

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 165MB/s]


AIモデルの準備が完了しました。

もし 'torch' や 'torchvision' がないというエラーが出たら、!pip install torch torchvision を実行してください。
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d6a60249bebf9bbee9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# ================================================================
# 2コマ目・改：AIの思考を覗く！リアルタイム物体認識
# ================================================================

# ----------------------------------------------------------------
# ステップ1＆2：準備（初回と同じ）
# ----------------------------------------------------------------
!pip install transformers torch gradio --quiet
from transformers import pipeline
import gradio as gr

# ----------------------------------------------------------------
# ステップ3：AIモデルの読み込み（初回と同じ）
# ----------------------------------------------------------------
image_classifier = pipeline("image-classification", model="google/vit-base-patch16-224")

# ----------------------------------------------------------------
# ステップ4：AIの「思考」を可視化する「命令」を作る
# ----------------------------------------------------------------
# 予測結果のリストを、GradioのLabelコンポーネントが扱える形式（辞書）に変換する
# ----------------------------------------------------------------
def classify_image_realtime(image):
    # 画像が入力されていない場合はNoneを返す
    if image is None:
        return None

    predictions = image_classifier(image)

    # 予測結果を {ラベル名: 確信度スコア} の辞書形式に変換する
    # 上位3件だけを取り出す
    result_dict = {p['label']: p['score'] for p in predictions[:10]}
    return result_dict

# ----------------------------------------------------------------
# ステップ5：ライブカメラ対応の操作パネルを起動する
# ----------------------------------------------------------------
# inputsにWebカメラを指定し、live=True に設定することでリアルタイム処理が実現する
# ----------------------------------------------------------------
gr.Interface(fn=classify_image_realtime,
             inputs=gr.Image(sources=["webcam"], type="pil", label="Webカメラ"),
             outputs=gr.Label(num_top_classes=10, label="AIの認識結果（トップ10）"),
             title="👀 AIの思考を覗く！リアルタイム物体認識カメラ",
             description="Webカメラに身の回りのものを写してみてください。AIが何を認識しているか、その確信度と一緒にリアルタイムで表示します。",
             live=True).launch(share=True)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cuda:0


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7ea9aed341af83350e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 必要なライブラリをすべてインポート
import gradio as gr
from PIL import Image, ImageFilter, ImageDraw
import numpy as np
import torch
import torchvision.transforms.functional as F
from torchvision.models import resnet50
import requests
from transformers import pipeline

# --- AIモデルとラベルの準備 ---
# この準備は一度だけ行います
try:
    # Image Classification model
    image_classifier = pipeline("image-classification", model="google/vit-base-patch16-224")
    # ResNet50 for image analysis
    model = resnet50(pretrained=True)
    model.eval()

    response = requests.get("https://git.io/JJkYN")
    labels = response.text.split("\n")
    print("AIモデルの準備が完了しました。")
except Exception as e:
    print(f"AIモデルの準備中にエラーが発生しました: {e}")
    image_classifier = None
    model = None
    labels = None

# --- 1. 画像加工のロジック ---
def image_processing(img, filter_type, blur_level, watermark_text):
    if img is None:
        return None, "画像をアップロードしてください。"
    processed_img = img.copy()
    if filter_type == "白黒":
        processed_img = processed_img.convert('L')
    elif filter_type == "90度回転":
        processed_img = processed_img.rotate(90, expand=True)
    if blur_level > 0:
        processed_img = processed_img.filter(ImageFilter.GaussianBlur(radius=blur_level))
    if watermark_text:
        draw = ImageDraw.Draw(processed_img)
        draw.text((10, 10), watermark_text, fill="white", stroke_width=1, stroke_fill="black")
    return processed_img, f"処理完了：{filter_type}"

# --- 2. AIによる画像分析のロジック ---
@torch.no_grad()
def predict(input_img):
    if model is None or labels is None or input_img is None:
        return "AIモデルが準備できていないか、画像がありません。"

    img = F.resize(input_img, (224, 224))
    img = F.to_tensor(img)
    if img.shape[0] == 1: # 白黒画像の場合
        img = img.repeat(3, 1, 1)
    img = F.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img = img.unsqueeze(0)

    output = model(img).squeeze(0)
    probs = torch.nn.functional.softmax(output, dim=0).numpy()

    return {labels[i]: float(probs[i]) for i in range(1000)}

# --- 3. AIの「思考」を可視化する「命令」を作る (リアルタイム物体認識) ---
def classify_image_realtime(image):
    if image_classifier is None or image is None:
        return "AIモデルが準備できていないか、画像がありません。"

    predictions = image_classifier(image)

    result_dict = {p['label']: p['score'] for p in predictions[:10]}
    return result_dict

# --- GradioのUIを構築 ---
with gr.Blocks() as demo:
    gr.Markdown("# 総合画像処理アプリ")
    gr.Markdown("手動での画像加工、AIによる画像分析、リアルタイム物体認識ができます。")

    with gr.Tabs():
        # --- タブ1: 画像加工 ---
        with gr.TabItem("画像加工"):
            with gr.Row():
                with gr.Column():
                    proc_input = gr.Image(type="pil", label="加工したい画像をアップロード")
                    filter_radio = gr.Radio(["なし", "白黒", "90度回転"], label="フィルターを選択", value="なし")
                    blur_slider = gr.Slider(0, 10, value=0, label="ぼかしの強さ")
                    watermark_input = gr.Textbox(label="透かし文字")
                    proc_btn = gr.Button("画像処理を実行")
                with gr.Column():
                    proc_output = gr.Image(label="加工後の画像")
                    proc_status = gr.Textbox(label="処理ステータス")
            proc_btn.click(
                fn=image_processing,
                inputs=[proc_input, filter_radio, blur_slider, watermark_input],
                outputs=[proc_output, proc_status]
            )

        # --- タブ2: AI画像分析 ---
        with gr.TabItem("AI画像分析"):
            with gr.Row():
                with gr.Column():
                    ai_input = gr.Image(type="pil", label="分析したい画像をアップロード")
                    ai_btn = gr.Button("AI分析を実行")
                with gr.Column():
                    ai_output = gr.Label(num_top_classes=5, label="分析結果")
            ai_btn.click(
                fn=predict,
                inputs=ai_input,
                outputs=ai_output
            )

        # --- タブ3: リアルタイム物体認識 ---
        with gr.TabItem("リアルタイム物体認識"):
             with gr.Row():
                 with gr.Column():
                     realtime_input = gr.Image(sources=["webcam"], type="pil", label="Webカメラ")
                     realtime_btn = gr.Button("AI分析を実行 (リアルタイム)") # ボタンは不要だが、UIを合わせるために残す
                 with gr.Column():
                     realtime_output = gr.Label(num_top_classes=10, label="AIの認識結果（トップ10）")
             # Use change() for real-time updates from webcam
             realtime_input.change(
                 fn=classify_image_realtime,
                 inputs=realtime_input,
                 outputs=realtime_output,
             )


# 必要なライブラリのインストールを促すメッセージ
print("\nもし 'torch' や 'torchvision' がないというエラーが出たら、!pip install torch torchvision を実行してください。")
# アプリケーションの起動
demo.launch(share=True)

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AIモデルの準備が完了しました。

もし 'torch' や 'torchvision' がないというエラーが出たら、!pip install torch torchvision を実行してください。
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://72b6a96381540bf61a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 必要なライブラリをインポートします
# gradio: Web UIを簡単に作るためのライブラリ
# PIL (Pillow): 画像を加工するためのライブラリ
# numpy: 数値計算、特にAIでよく使われるライブラリ
# その他: AIモデルを動かすためのライブラリ
import gradio as gr
from PIL import Image
import numpy as np
import requests
import torch
from torchvision.models import resnet50
import torchvision.transforms.functional as F

# --- ここからAIの準備 (この部分は完成しています) ---
# 授業で使ったResNet50モデルとImageNetラベルを準備します
# この部分は一度だけ実行されます
try:
    model = resnet50(pretrained=True)
    model.eval()
    response = requests.get("https://git.io/JJkYN")
    labels = response.text.split("\n")
    print("AIモデルの準備OK！")
except Exception as e:
    print("AIモデルの準備中にエラーが発生しました:", e)
    model = None
    labels = None
# --- AIの準備ここまで ---


# --- 関数を定義するエリア ---

# 【課題１】画像加工用の関数を完成させよう！
# 1コマ目の最後に作った「白黒フィルター」の処理を思い出して書いてみよう！
def image_filter(input_img):
    print("画像加工ボタンが押されました！")
    if input_img is None:
        return None # 画像がなければ何もしない

    # ▼▼▼ ここに、Pillowを使った画像処理コードを追加しよう ▼▼▼
    # ヒント: input_img.convert('L') を使うと白黒にできます

    processed_img = input_img.convert('L') # 今はまだ何もしないので、この行を書き換えよう！

    # ▲▲▲ ここまで ▲▲▲

    return processed_img


# 【課題２】AI分析用の関数を完成させよう！
# 2コマ目に体験した「AIによる画像判定」のコードを参考に、中身を完成させよう！
def predict_image(input_img):
    print("AI分析ボタンが押されました！")
    if model is None or labels is None or input_img is None:
        return "AIモデルが準備できていないか、画像がありません。"

    # ▼▼▼ ここに、AIに画像を分析させるコードを追加しよう ▼▼▼
    # ヒント: 教材『3.Gradio(Webインタフェース)の基本.ipynb』の最後の部分を参考にしよう
    # 手順１：画像の前処理 (リサイズ、Tensorに変換、正規化)
    # 手順２：AIによる推論 (model(img) を呼び出す)
    # 手順３：結果を整形して、returnで返す
    img = F.resize(input_img, (224, 224))
    img = F.to_tensor(img)
    if img.shape[0] == 1: # 白黒画像の場合
        img = img.repeat(3, 1, 1)
    img = F.normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    img = img.unsqueeze(0)

    # 推論の実行
    with torch.no_grad():
        output = model(img).squeeze(0)
    probs = torch.nn.functional.softmax(output, dim=0).numpy()

    # 結果を{ラベル: 確率}の辞書形式で返す
    result = {labels[i]: float(probs[i]) for i in range(1000)} # 今はまだダミーのデータなので、この行を書き換えよう！

    # ▲▲▲ ここまで ▲▲▲

    return result


# --- Web UIを作るエリア (この部分は完成しています) ---
# gr.Blocks() を使うと、より自由にUIを組み立てられます
with gr.Blocks() as demo:
    # アプリ全体のタイトル
    gr.Markdown("# オリジナル画像加工＆AI分析アプリ")

    # gr.Tabs() で機能を切り替えるタブを作ります
    with gr.Tabs():
        # 1つ目のタブ
        with gr.TabItem("画像加工フィルター"):
            gr.Markdown("## 好きな画像を加工してみよう！")
            with gr.Row():
                # 左側に配置するUI部品
                with gr.Column():
                    image_input_filter = gr.Image(type="pil", label="画像をアップロード")
                    process_button = gr.Button("加工する！")
                # 右側に配置するUI部品
                with gr.Column():
                    image_output_filter = gr.Image(label="加工後の画像")

            # ボタンが押されたら、どの関数を呼び出すか設定
            process_button.click(fn=image_filter, inputs=image_input_filter, outputs=image_output_filter)

        # 2つ目のタブ
        with gr.TabItem("AI画像分析"):
            gr.Markdown("## AIが写真に写っているものを当てるよ！")
            with gr.Row():
                # 左側に配置するUI部品
                with gr.Column():
                    image_input_ai = gr.Image(type="pil", label="画像をアップロード")
                    predict_button = gr.Button("AIに分析してもらう！")
                # 右側に配置するUI部品
                with gr.Column():
                    # gr.LabelはAIの分析結果を表示するのに便利です
                    label_output_ai = gr.Label(num_top_classes=3, label="分析結果")

            # ボタンが押されたら、どの関数を呼び出すか設定
            predict_button.click(fn=predict_image, inputs=image_input_ai, outputs=label_output_ai)


# アプリを起動します
print("アプリを起動します。URLが表示されたらクリックしてください。")
demo.launch()

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AIモデルの準備OK！
アプリを起動します。URLが表示されたらクリックしてください。
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a6ecd06fab5fc0e1ee.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
